In [1]:
# %pip install llama-index-readers-wikipedia
# %pip install llama-index
%pip install wikipedia
# %pip install faiss-cpu
# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-embeddings-instructor
# %pip install llama-index-llms-llama-cpp

# %pip install llama-index-vector-stores-faiss


Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.readers.wikipedia import WikipediaReader


In [3]:
reader = WikipediaReader()

In [4]:
cities = ["จังหวัดอยุธยา",
        #   "จังหวัดปทุมธานี",
        #   "จังหวัดนครราชสีมา",
        #   "จังหวัดเชียงใหม่",
          "จังหวัดชลบุรี",
          "จังหวัดนครศรีธรรมราช",]

In [5]:
import wikipedia
wikipedia.set_lang('th')

In [6]:
documents = reader.load_data(pages=cities)

In [7]:
documents

[Document(id_='4203', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='พระนครศรีอยุธยา เป็นจังหวัดหนึ่งในภาคกลางและเป็นเขตเศรษฐกิจอุตสาหกรรมที่สำคัญ โดยมีผลิตภัณฑ์มวลรวมของจังหวัดมีมูลค่าสูงเป็นอันดับ 3 ของประเทศ และมีประวัติศาสตร์ความเป็นมาที่ยาวนาน เคยมีชื่อเสียงเป็นแหล่งปลูกข้าวที่สำคัญ จังหวัดพระนครศรีอยุธยาเป็นจังหวัดเดียวในประเทศไทยที่ไม่มีอำเภอเมือง แต่มี อำเภอพระนครศรีอยุธยา เป็นศูนย์กลางการบริหารจัดการด้านต่าง ๆ ชาวบ้านโดยทั่วไปนิยมเรียกอีกชื่อว่า "กรุงเก่า" หรือ "เมืองกรุงเก่า" ห่างจากกรุงเทพมหานครประมาณ 75 กิโลเมตร\n\n\n== ประวัติศาสตร์ ==\n\nพระนครศรีอยุธยาเคยเป็นราชธานี (เมืองหลวง) ของอาณาจักรอยุธยา หรืออาณาจักรสยาม ตลอดระยะเวลา 417 ปี ตั้งแต่ พ.ศ. 1893 กระทั่งเสียกรุงแก่พม่า เมื่อ พ.ศ. 2310 ครั้นเมื่อสมเด็จพระเจ้ากรุงธนบุรีทรงสถาปนาราชธานีแห่งใหม่ที่กรุงธนบุรี กรุงศรีอยุธยาก็ไม่ได้กลายเป็นเ

In [8]:
import faiss

# dimensions of BAAI/bge-m3
d = 1024
faiss_index = faiss.IndexFlatIP(d)

In [9]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [10]:
embedding_model_name = '../../bge-m3'
# embedding_model_name = 'kornwtp/simcse-model-phayathaibert'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024, device='cpu')

In [11]:
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
)        
from llama_index.vector_stores.faiss import FaissVectorStore

In [12]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=256,
    chunk_overlap=10,
    separator=" ",
)

In [13]:
nodes = splitter.get_nodes_from_documents(
    documents, show_progress=True
)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
nodes

[TextNode(id_='d041e839-a99f-4c44-b02b-ee17c93222e8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4203', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='915a148c9b95fb7f707aeb08398cfe8c6f1641975d0c519dabd329a9f68cb9d8'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='e8a88f11-75aa-4058-b81e-c24815f6635a', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='512a1e78b324df81a579f05d814477a8f486a5654e5509e1efeac3686a1f1427')}, metadata_template='{key}: {value}', metadata_separator='\n', text='พระนครศรีอยุธยา เป็นจังหวัดหนึ่งในภาคกลางและเป็นเขตเศรษฐกิจอุตสาหกรรมที่สำคัญ โดยมีผลิตภัณฑ์มวลรวมของจังหวัดมีมูลค่าสูงเป็นอันดับ 3 ของประเทศ และมีประวัติศาสตร์ความเป็นมาที่ยาวนาน เคยมีชื่อเสียงเป็นแหล่งปลูกข้าวที่สำคัญ', mimetype='text/plain', start_char_idx=0, end_char_idx=219, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_=

In [15]:
from llama_index.core import Settings

Settings.llm = None
Settings.embed_model = embed_model

LLM is explicitly disabled. Using MockLLM.


In [16]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes=nodes,storage_context=storage_context, show_progress=True)

Generating embeddings:   0%|          | 0/195 [00:00<?, ?it/s]

In [17]:
index.storage_context.persist(persist_dir='../../index')
